In [ ]:
# | default_exp ds.tplot.trans
# | export
from pydantic import (
    BaseModel,
    ConfigDict,
    model_validator,
)

import pyspedas
from pytplot import options, split_vec, join_vec
from space_analysis.ds.ts import B_TsOption
from pyspedas.cotrans.minvar_matrix_make import minvar_matrix_make
from pyspedas import tvector_rotate
from pyspedas.analysis.tvectot import tvectot
import sys

from typing import Callable

In [ ]:
# | export
# if "slice-1" in config.trans:
#     tvar = split_vec(tvar)[:1]
# if "slice-3" in config.trans:
#     tvar = join_vec(split_vec(tvar)[:3])


def magnitude(tvar: str):
    tvar2plot = tvectot(tvar, join_component=False)
    options(tvar2plot, "legend_names", None)
    return tvar2plot


def mva(tvar: str):
    minvar_matrix_make(tvar)
    tvar2plot = tvector_rotate(tvar + "_mva_mat", tvar)[0]
    return tvar2plot


def magnitude_join(tvar: str):
    tvar2plot = tvectot(tvar, join_component=True)
    return tvar2plot


def mva_rename(tvar: str, legend_names=B_TsOption["legend_names"]):
    options(tvar, "legend_names", legend_names)
    return tvar

In [ ]:
# | export
class TransformOp(BaseModel):
    """Transformation operation"""

    # allow addtional fields
    model_config = ConfigDict(extra="allow")

    name: str

    transform_func: Callable[[str], str] = None

    @model_validator(mode="after")
    def map_transform_func(self):
        self.transform_func = transform_func(self.name)
        return self


def transform_func(name):
    transform_func_maps = {
        "avg": pyspedas.avg_data,
        "slice-1": lambda tvar: split_vec(tvar)[:1],
        "slice-3": lambda tvar: join_vec(split_vec(tvar)[:3]),
    }
    func = transform_func_maps.get(name, None) or getattr(sys.modules[__name__], name)
    return func